part2 classification

red dataset

In [0]:
import torch 
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torchvision.datasets as datasets
from torch.autograd import Variable
import matplotlib.pyplot as plt
from PIL import Image
import os
import copy
import torch.utils.data as data_utils
import requests, zipfile, io
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap


# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 40
num_classes = 10
batch_size = 8
learning_rate = 0.0001

# ECE5470 dataset
url = 'https://www.via.cornell.edu/ece5470/Lab9data2r.zip'
r=requests.get(url).content

#save data in data dir
z = zipfile.ZipFile(io.BytesIO(r))
# os.mkdir('data1')
z.extractall('./data1')

In [0]:
""" Custom datatset loader
    based on https://github.com/utkuozbulak/pytorch-custom-dataset-examples
"""
import pandas as pd
import imageio

class SimpleDataset():
    def __init__(self, data_path, csv_name, transform = None ):
        """
        Args:
            data_path (string): path to the folder where images and csv files are located
            csv_name (string): name of the csv lablel file
            transform: pytorch transforms for transforms and tensor conversion
        """
        # Set path
        self.data_path = data_path
        
        # Read the csv file
        self.data_info = pd.read_csv(data_path + csv_name, header=0)
       
        # First column contains the image paths
        self.image_arr = np.asarray(self.data_info.iloc[:, 0])
        
        # Second column is the labels
        self.label_arr = np.asarray(self.data_info.iloc[:, 1])
        # Calculate len
        self.data_len = len(self.data_info.index)
        
    def __getitem__(self, index):
        # Get image name from the pandas df
        single_image_name = self.image_arr[index]
        # Open image
        a=single_image_name[:-3] + "png"
        single_image_name=a
        img_as_img = imageio.imread(self.data_path + single_image_name)
        # Open segmented mask
        a = single_image_name[:2] + "seg" + single_image_name[2:]
        imseg_name = a[:-3] + "png"
        imgsg_as_img = imageio.imread(self.data_path + imseg_name)
        
        
        # Get label(class) of the image based on the cropped pandas column
        single_image_label = self.label_arr[index]
        
        return (img_as_img, single_image_label, imgsg_as_img)

    def __len__(self):
        return self.data_len

In [3]:
mydata = SimpleDataset( "./data1/", "labels.csv")

#splitting into images and labels 
X  = []
y  = []
Xs = []
for i in range(len(mydata)):
    X.append(mydata[i][0])
    y.append((mydata[i][1]))
    Xs.append(mydata[i][2])


#converting into numpy arrays to enable easy reshaping and other array operations
    
X  = np.asarray(X)
Xs = np.asarray(Xs)
print("Shape of the input image", X.shape, Xs.shape)
y= np.asarray(y)

X  = np.swapaxes(X,1,3)
X  = np.swapaxes(X,2,3)
print("Shape of the input image", X.shape, Xs.shape,y.shape)

Shape of the input image (5000, 32, 32, 3) (5000, 32, 32)
Shape of the input image (5000, 3, 32, 32) (5000, 32, 32) (5000,)


In [4]:
Xtr=X[:3500,:,:,:]/255
Xstr=Xs[:3500,:,:]/255
ytr=y[:3500]

Xval=X[3500:4000,:,:,:]/255 #need to change validation size for 32x32 & 64x64 images
Xsval=Xs[3500:4000,:,:]/255
yval=y[3500:4000]

Xte=X[4000:5000,:,:,:]/255 #need to change test size for 32x32 & 64x64 images
Xste=Xs[4000:5000,:,:]/255
yte=y[4000:5000]


print(Xtr.shape,Xstr.shape,ytr.shape)
print(Xval.shape,Xsval.shape,yval.shape)
print(Xte.shape,Xste.shape,yte.shape)

(3500, 3, 32, 32) (3500, 32, 32) (3500,)
(500, 3, 32, 32) (500, 32, 32) (500,)
(1000, 3, 32, 32) (1000, 32, 32) (1000,)


In [5]:
batch_size = 8
'''
inputs and segs are your data. 
'''
targets=ytr

inputs = torch.from_numpy(Xtr).float()
targets = torch.from_numpy(targets).float()
segs    = torch.from_numpy(Xstr).float()

print('Size of inputs: {}'. format(inputs.shape))
print('Size of ground truth: {}'. format(segs.shape))
print(targets.shape)

# Dataloader
trainset = data_utils.TensorDataset(inputs, segs, targets)
train_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size= batch_size, 
                                          shuffle=True,drop_last=True)

Size of inputs: torch.Size([3500, 3, 32, 32])
Size of ground truth: torch.Size([3500, 32, 32])
torch.Size([3500])


In [6]:
targets=yte
inputs = torch.from_numpy(Xte).float()
targets = torch.from_numpy(targets).float()
segs    = torch.from_numpy(Xste).float()

print('Size of inputs: {}'. format(inputs.shape))
print('Size of ground truth: {}'. format(segs.shape))
print(targets.shape)

# Dataloader
testset = data_utils.TensorDataset(inputs, segs,targets)
test_loader = torch.utils.data.DataLoader(testset, batch_size= batch_size, 
                                          shuffle=False, drop_last=True)

Size of inputs: torch.Size([1000, 3, 32, 32])
Size of ground truth: torch.Size([1000, 32, 32])
torch.Size([1000])


In [0]:
# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(2*32*32, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

model = ConvNet(num_classes).to(device)

In [0]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Train the model
total_step = len(train_loader)
resultstr = []
resultsval = []
correct = 0
total = 0
model.train()
for epoch in range(num_epochs):
    for i, (images, segs, targets) in enumerate(train_loader):
        segs = segs.to(device)
        segs = torch.reshape(segs, (batch_size,1,32,32))
        labels = targets.to(device)
        
        # Forward pass
        outputs = model(images.cuda())
        loss = criterion(outputs, labels.to(torch.long))
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # O
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.to(torch.int) == labels.to(torch.int)).sum().item()

        if (i+1) % 50 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], TrLoss: {:.4f}' 
                   .format(epoch+1,num_epochs,i+1, total_step, loss.data))
    
    resultstr.append(loss.data.cpu().to(torch.float))
acc = correct/total
print('Training accuracy: {} %'.format(100*acc))

Epoch [1/40], Step [50/437], TrLoss: 2.4010
Epoch [1/40], Step [100/437], TrLoss: 2.4736
Epoch [1/40], Step [150/437], TrLoss: 2.0550
Epoch [1/40], Step [200/437], TrLoss: 2.1515
Epoch [1/40], Step [250/437], TrLoss: 2.3487
Epoch [1/40], Step [300/437], TrLoss: 2.2108
Epoch [1/40], Step [350/437], TrLoss: 2.4453
Epoch [1/40], Step [400/437], TrLoss: 2.1873
Epoch [2/40], Step [50/437], TrLoss: 2.3154
Epoch [2/40], Step [100/437], TrLoss: 2.3636
Epoch [2/40], Step [150/437], TrLoss: 2.3027
Epoch [2/40], Step [200/437], TrLoss: 2.5069
Epoch [2/40], Step [250/437], TrLoss: 2.0324
Epoch [2/40], Step [300/437], TrLoss: 2.3521
Epoch [2/40], Step [350/437], TrLoss: 2.2464
Epoch [2/40], Step [400/437], TrLoss: 2.2533
Epoch [3/40], Step [50/437], TrLoss: 2.0478
Epoch [3/40], Step [100/437], TrLoss: 2.2675
Epoch [3/40], Step [150/437], TrLoss: 2.2281
Epoch [3/40], Step [200/437], TrLoss: 2.2222
Epoch [3/40], Step [250/437], TrLoss: 2.3322
Epoch [3/40], Step [300/437], TrLoss: 2.2949
Epoch [3/40],

In [10]:
# Test the model
model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    OP = []
    PR = []
    LB = []
    for i, (images, segs, targets) in enumerate(test_loader):
        segs = segs.to(device)
        segs = torch.reshape(segs, (batch_size,1,32,32))
        labels = targets.to(device)
        LB.append(labels.cpu().numpy())
        outputs=model(images.cuda())
        OP.append(outputs.cpu().numpy())
        _, predicted = torch.max(outputs.data, 1)
        PR.append(predicted.cpu().numpy())
        total += labels.size(0)
        correct += (predicted.to(torch.int) == labels.to(torch.int)).sum().item()

print('Test Accuracy of the model on the 1000 test images: {} %'.format(100 * correct / total))

Test Accuracy of the model on the 1000 test images: 46.8 %


In [11]:
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

pred = np.array(PR)
pred = pred.reshape(1000)
print(np.shape(pred))
label = np.array(LB)
label = label.reshape(1000)
print(np.shape(label))
cfm=confusion_matrix(pred, label)
print(cfm)


(1000,)
(1000,)
[[48  0  5  3  2  3  6  3  4  1]
 [ 2 90  2  6  6  0  2  6  4  2]
 [ 5  1 25 11  2  4  2  4  2  4]
 [ 1  0 19 53  4 23  1  6 12  6]
 [21 16 10  4 64  6 21 10  8 22]
 [ 0  2 11  7  1 22  4  0  4  2]
 [10  1  7  1  7  0 48  1  8  7]
 [ 6  6  6  2  2  1  2 46  2  9]
 [ 8  1 13  9  3  6  2  3 34 10]
 [ 6  8  8 10 14  6  6 16 12 38]]
